In [1]:
import sys
sys.path.append('../analysis_v2/')
import SiemensQuadraProperties as sqp
import CoincidenceGeneration as cg
import SimulationDataset as sd
import PhysicsConstants as pc
import GATEfileWriter as gf

import time

import matplotlib.pyplot as plt
params = {'legend.fontsize': 15,
          'legend.title_fontsize': 15,
          'legend.loc': "upper left",
          'axes.labelsize': 15,
          'xtick.labelsize': 15,
          'ytick.labelsize': 15}
plt.rcParams.update(params)

# Fix random seed for reproducibility, or leave blank to allow the results to vary
import numpy as np
RNG = np.random.default_rng(1234)

# Set batch size for coincidence generation
# Larger batches will use more RAM, and will eventually become inefficient due to wasted events
# Smaller batches are inefficient due to overheads
# 256-1024 seems to be about right
BATCH_SIZE=1024

Using an analogous structure to the NECR notebook, but this just creates a single data sample, and then saves it in a GATE-compatible ROOT file

In [2]:
# Simulation parameters
detectorLength = 1024
phantomLength = 700
datasetSize = 1000000
siemensEmin = 435.0
siemensEmax = 585.0
detectorMaterial = "LSO"
timeSec = 0
fileName = "testReconstructionData"

# Check if sim already done
import os
if os.path.isfile( fileName + ".root" ) and os.path.isfile( fileName + ".hroot" ):
    print( "Skipping simulation since output files (" + fileName + ") exist" )
else:
    start = time.time_ns()
    
    # Make data samples
    # Have to use the full-detail (Crystal) version of the geometry to get the GATE module IDs
    # 1) High activity offset
    tracer1Data = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "LinearF18", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True,
                                   SourceOffset=30, STIRheader=fileName+".hroot" )
    tracer1Activity = pc.TracerActivityAtTime( 1100E5, timeSec, "F18" )
    
    # 2) low activity central
    tracer2Data = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "LinearF18", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True,
                                   SourceOffset=0 )
    tracer2Activity = pc.TracerActivityAtTime( 400E5, timeSec, "F18" )
    
    # 3) Intrinsic background
    crystalData = sd.CreateDataset( detectorLength, "SiemensCrystal", phantomLength, "Siemens", datasetSize, siemensEmin, siemensEmax, detectorMaterial, UseNumpy=True )
    crystalActivity = sqp.Lu176decaysInMass( sqp.DetectorMass(detectorMaterial) )
    
    # Create a moduleID lookup table
    moduleIDs = tracer1Data.GetModuleIDs()
    moduleIDs.update( tracer2Data.GetModuleIDs() )
    moduleIDs.update( crystalData.GetModuleIDs() )
    
    end = time.time_ns()
    print( "Photon simulation (if needed) and data loading: " + str( (end-start)/1e9 ) + "s" )
    start = time.time_ns()
    
    # Assemble data sources, including crystal intrinsic background
    activityList = [ tracer1Activity, tracer2Activity, crystalActivity ]
    dataList = [ tracer1Data, tracer2Data, crystalData ]
    
    # Create a generator object that will simulate radioactive decays and load Geant4 events
    generator = cg.GenerateCoincidences( BATCH_SIZE, activityList, dataList, RNG, CoincidenceWindow=4.7, SimulationWindow=2E9, \
                                         MultiWindow=False, EnergyResolution=0.0, EnergyMin=0.0, EnergyMax=0.0, TimeResolution=0.0 )
    
    # Save photons to a GATE format file
    gf.GATEfromGenerator( BATCH_SIZE, generator, fileName+".root", moduleIDs, sqp.DetectorRadius(), PairMode="Exclusive", ZMin=-325.0, ZMax=325 )
    
    end = time.time_ns()
    print( "Coincidence generation and writing: " + str( (end-start)/1e9 ) + "s" )

Skipping simulation since output files (testReconstructionData) exist


In [3]:
import STIRreconstruction as sr

sinogramName = "testSino"
sr.CreateSinogram( fileName, sinogramName )

Number of rings
Number of detectors per ring
Inner ring diameter (cm)
Average depth of interaction (cm)
Distance between rings (cm)
Default bin size (cm)
View offset (degrees)
Maximum number of non-arc-corrected bins
Default number of arc-corrected bins
number of submodules_Y
number of submodules_Z
number of crystals_Z
number of crystals_Y
Number of rings
number of crystals_Y

INFO: InputStreamFromROOTFile: Processing data with the following event type inclusions:
  Unscattered from same eventID:        1
  Unscattered from different eventIDs:  0
  Scattered from same eventID:          0
  Scattered different eventIDs:         0

INFO: CListModeDataROOT: TOF information is missing. Set relevant keywords if you need TOF:
	Maximum number of (unmashed) TOF time bins
	Size of unmashed TOF time bins (ps)
	TOF timing resolution (ps)

INFO: CListModeDataROOT: used ROOT file testReconstructionData.root

INFO: Setting energy window keys as in STIR3.0

INFO: Interfile warning: I did not recognis







LmToProjData NOT Using FRAME_BASED_DT_CORR
lm to projdata parameters :=
input file := testReconstructionData.hroot
template projdata := testReconstructionData.hs
frame definition file := 
num events to store := -1
output filename prefix := ./testSino
bin normalisation type for pre-normalisation := None

bin normalisation type for post-normalisation := None

maximum absolute segment number to process := 0
do pre normalisation := 0
num tof bins in memory := 1
num segments in memory := 1
store prompts := 1
store delayeds := 1
list event coordinates := 0
end := 

Processing time frame 1


12000000 events stored


Number of prompts stored in this time period : 12423700
Number of delayeds stored in this time period: 0
Last stored event was recorded before time-tick at 0 secs
Total number of counts (either prompts/trues/delayeds) stored: 12423700

This took 7.15s CPU time.
